# Add brunel jar

In [1]:
%Addjar --magic https://brunelvis.org/jar/spark-kernel-brunel-all-2.3.jar -f 

Starting download from https://brunelvis.org/jar/spark-kernel-brunel-all-2.3.jar
Finished download of spark-kernel-brunel-all-2.3.jar


In [2]:

// This function is used to setup the access of Spark to your Object Storage. The definition contains your credentials.
// You might want to remove those credentials before you share your notebook.
def setHadoopConfig64e9c4bd3e4148978db0a312dfcc0a93(name: String) = {
    // This function sets the Hadoop configuration so it is possible to
    // access data from Bluemix Object Storage using Spark

    val prefix = "fs.swift.service." + name
    sc.hadoopConfiguration.set(prefix + ".auth.url", "https://identity.open.softlayer.com" + "/v3/auth/tokens")
    sc.hadoopConfiguration.set(prefix + ".auth.endpoint.prefix","endpoints")
    sc.hadoopConfiguration.set(prefix + ".tenant", "419cd8dece644c82af5a615b62af38e1")
    sc.hadoopConfiguration.set(prefix + ".username", "20ae3d743b904885bcebe7b72c6faeb3")
    sc.hadoopConfiguration.set(prefix + ".password", "s9b#uo2T(T_NoZe(")
    sc.hadoopConfiguration.setInt(prefix + ".http.port", 8080)
    sc.hadoopConfiguration.set(prefix + ".region", "dallas")
    sc.hadoopConfiguration.setBoolean(prefix + ".public", false)
}

// you can choose any name
val name = "keystone"
setHadoopConfig64e9c4bd3e4148978db0a312dfcc0a93(name)

val sqlContext = new org.apache.spark.sql.SQLContext(sc)

val dfData1 = sqlContext.
    read.format("com.databricks.spark.csv").
    option("header", "true").
    option("inferSchema", "true").
    load("swift://BankChurnAnalysis." + name + "/Churn_Rate.csv")
dfData1.show(5)


+----+-------+------------+----------+
|Year|Quarter|Quarter_Year|Churn_Rate|
+----+-------+------------+----------+
|2014|      1|        1Q14|      18.1|
|2014|      2|        2Q14|      18.7|
|2014|      3|        3Q14|      19.3|
|2014|      4|        4Q14|      19.9|
|2015|      1|        1Q15|      20.5|
+----+-------+------------+----------+
only showing top 5 rows



In [3]:
%%brunel data('dfData1') x(QUARTER_YEAR) y(CHURN_RATE) bar tooltip(#all) sort(YEAR:ascending, QUARTER:ascending) tooltip(#all) axes(x:'Time', y:'Churn Rate':grid) 
:: width=800, height=500

#  Churn Data Analysis

In [4]:
val dfData2 = sqlContext.
    read.format("com.databricks.spark.csv").
    option("header", "true").
    option("inferSchema", "true").
    load("swift://BankChurnAnalysis." + name + "/CUST_SUM.csv")
dfData2.show(5)


+----------+---+---+---------+----------+-------+--------+-----+--------+------------+-------+----------+---------+-----+----------------+---------+-----------+
|   CUST_ID|SEX|AGE|EDUCATION|INVESTMENT| INCOME|ACTIVITY|CHURN|YRLY_AMT|AVG_DAILY_TX|YRLY_TX|AVG_TX_AMT|NEGTWEETS|STATE| EDUCATION_GROUP|TwitterID|CHURN_LABEL|
+----------+---+---+---------+----------+-------+--------+-----+--------+------------+-------+----------+---------+-----+----------------+---------+-----------+
|1009530860|  F| 84|        2|    114368|3852862|       5|    0|700259.0|    0.917808|    335|   2090.32|        3|   TX|Bachelors degree|        0|      false|
|1009544000|  F| 44|        2|     90298|3849843|       1|    0|726977.0|    0.950685|    347|   2095.04|        2|   CA|Bachelors degree|        0|      false|
|1009534260|  F| 23|        2|     94881|3217364|       1|    1|579084.0|    0.920548|    336|   1723.46|        5|   CA|Bachelors degree|        0|       true|
|1009574010|  F| 24|        2|    

In [15]:
%%brunel data('dfData2') x(STATE) y(INCOME) mean(INCOME) bar tooltip(#all) sort(INCOME:ascending) tooltip(#all) axes(x:'State', y:'Income':grid) 
:: width=800, height=500

In [5]:
%%brunel data('dfData2') map key(STATE) x(STATE) color(INCOME) mean(INCOME) label(STATE) tooltip(#all) :: width=800, height=500

In [6]:
%%brunel data('dfData2') 
         x(AGE) y(#count:linear) color(CHURN_LABEL) bin(AGE) interaction(select) stack bar tooltip(#all) filter(CHURN_LABEL) legends(none) |
         x(AVG_DAILY_TX) y(#count:linear) color(CHURN_LABEL) opacity(#selection) bin(AVG_DAILY_TX) stack bar tooltip(#all) axes(x:10:'AVG DAILY TX', y)|
         x(AVG_TX_AMT) y(#count:linear) color(CHURN_LABEL) opacity(#selection) bin(AVG_TX_AMT) stack bar tooltip(#all)  axes(x,y) legends(none) |
        x(INCOME) y(#count:linear) color(CHURN_LABEL) opacity(#selection) bin(INCOME) stack bar tooltip(#all) tooltip(#all) axes(x,y) legends(none)
:: width=800, height=600

In [7]:

%%brunel data('dfData2')
         x(SEX) y(#count:linear) color(CHURN_LABEL) stack bar tooltip(#all) sort(SEX) interaction(select) filter(CHURN_LABEL) axes(x:'GENDER', y) legends(none) |
         x(ACTIVITY) y(#count:linear) color(CHURN_LABEL) stack bar tooltip(#all) sort(ACTIVITY) opacity(#selection)  | 
         x(EDUCATION_GROUP) y(#count:linear) color(CHURN_LABEL) stack bar tooltip(#all) sort(#count) opacity(#selection)  axes(x:'',y) legends(none)
:: width=900, height=600

In [8]:
%%brunel data('dfData2') map key(STATE) x(STATE) color(CHURN) mean(CHURN) label(STATE) tooltip(#all) :: width=800, height=500